<a href="https://colab.research.google.com/github/TheVijay-15/GlobalSuperstore-Sales-Forecasting-Analytics/blob/main/02_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing & Feature Engineering
## Goals:
## 1. Handle missing values
## 2. Create new features for ML models
## 3. Prepare data for time series forecasting

In [ ]:
# import libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load the initial cleaned data
df = pd.read_csv('/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/initial_cleaned.csv')

In [ ]:
# 1. Handle Missing Values
print("Handling missing values...")

# Postal Code has missing values
# Fill with 0 and create a flag
df['Postal_Code_Missing'] = df['Postal Code'].isna().astype(int)
df['Postal Code'].fillna(0, inplace=True)

# Check if any other missing values remain
print(f"Missing values after handling: {df.isnull().sum().sum()}")

Handling missing values...
Missing values after handling: 0


###The current approach of filling with 0 (and creating a Postal_Code_Missing flag) is a common and often effective way to handle missing categorical or quasi-numerical identifiers, as it allows the model to learn from the absence of the postal code.

In [ ]:
# 2. Create Time-Based Features
print("\nCreating time-based features...")

# Ensure date columns are datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

# Time difference features
df['Shipping_Days'] = (df['Ship Date'] - df['Order Date']).dt.days
df['Processing_Time_Category'] = pd.cut(df['Shipping_Days'],
                                        bins=[0, 3, 7, 14, 100],
                                        labels=['Express (0-3)', 'Fast (4-7)',
                                                'Standard (8-14)', 'Slow (15+)'])

# Cyclical time features for seasonality
df['Order_Month_Sin'] = np.sin(2 * np.pi * df['Order Date'].dt.month/12)
df['Order_Month_Cos'] = np.cos(2 * np.pi * df['Order Date'].dt.month/12)
df['Order_Day_Sin'] = np.sin(2 * np.pi * df['Order Date'].dt.day/31)
df['Order_Day_Cos'] = np.cos(2 * np.pi * df['Order Date'].dt.day/31)

# Holiday periods
df['Is_Holiday_Season'] = df['Order Date'].dt.month.isin([11, 12]).astype(int)
df['Is_EndOfQuarter'] = df['Order Date'].dt.month.isin([3, 6, 9, 12]).astype(int)


Creating time-based features...


In [ ]:
# 3. Create Business Metrics Features
print("\nCreating business metrics features...")

# Profitability indicators
df['Profit_Margin'] = (df['Profit'] / df['Sales']).replace([np.inf, -np.inf], 0) * 100
df['Is_Profitable'] = (df['Profit'] > 0).astype(int)
df['Profit_Per_Unit'] = df['Profit'] / df['Quantity'].replace(0, 1)

# Efficiency metrics
df['Sales_per_Shipping_Cost'] = df['Sales'] / df['Shipping Cost'].replace(0, 1)
df['Discount_Effectiveness'] = (df['Sales'] * df['Discount']) / df['Profit'].replace(0, 1)

# Customer value metrics (will be aggregated later)
df['Unit_Price'] = df['Sales'] / df['Quantity'].replace(0, 1)


Creating business metrics features...


In [ ]:
# 4. Categorical Feature Encoding
print("\nEncoding categorical features...")

# Label encoding for ordinal categories
le = LabelEncoder()
categorical_cols = ['Ship Mode', 'Segment', 'Category', 'Sub-Category',
                    'Region', 'Market', 'Country']

for col in categorical_cols:
    if col in df.columns:
        df[f'{col}_Encoded'] = le.fit_transform(df[col].astype(str))

# One-hot encoding for important categoricals (limited to avoid curse of dimensionality)
# Exclude 'Segment' from one-hot encoding here to preserve the original column for later aggregations
df = pd.get_dummies(df, columns=['Ship Mode', 'Region'],
                    prefix=['Ship', 'Region'], drop_first=True)


Encoding categorical features...


In [ ]:
# 5. Create Aggregated Customer Features

print("\nCreating customer-level features...")

customer_features = df.groupby('Customer ID').agg({
    'Sales': ['sum', 'mean', 'count', 'std'],
    'Profit': ['sum', 'mean'],
    'Order ID': 'nunique',
    'Order Date': ['min', 'max']
}).round(2)

customer_features.columns = ['_'.join(col).strip() for col in customer_features.columns.values]
customer_features = customer_features.rename(columns={
    'Sales_sum': 'Customer_Total_Sales',
    'Sales_mean': 'Customer_Avg_Sale',
    'Sales_count': 'Customer_Transaction_Count',
    'Sales_std': 'Customer_Sales_Std',
    'Profit_sum': 'Customer_Total_Profit',
    'Profit_mean': 'Customer_Avg_Profit',
    'Order ID_nunique': 'Customer_Order_Count',
    'Order Date_min': 'Customer_First_Order',
    'Order Date_max': 'Customer_Last_Order'
})

# Calculate recency, frequency, monetary (RFM) values
customer_features['Customer_Recency'] = (pd.Timestamp('2015-01-01') -
                                         customer_features['Customer_Last_Order']).dt.days
customer_features['Customer_Frequency'] = customer_features['Customer_Order_Count']
customer_features['Customer_Monetary'] = customer_features['Customer_Total_Sales']
customer_features['Customer_Lifetime'] = (customer_features['Customer_Last_Order'] -
                                          customer_features['Customer_First_Order']).dt.days

# Merge back to main dataframe
df = df.merge(customer_features, on='Customer ID', how='left')


Creating customer-level features...


In [ ]:
# 6. Create Product Features
print("\nCreating product-level features...")

product_features = df.groupby('Product ID').agg({
    'Sales': ['sum', 'mean', 'count', 'std'],
    'Profit': ['sum', 'mean'],
    'Quantity': ['sum', 'mean'],
    'Discount': 'mean'
}).round(2)

product_features.columns = ['_'.join(col).strip() for col in product_features.columns.values]
product_features = product_features.rename(columns={
    'Sales_sum': 'Product_Total_Sales',
    'Sales_mean': 'Product_Avg_Sale',
    'Sales_count': 'Product_Transaction_Count',
    'Sales_std': 'Product_Sales_Std',
    'Profit_sum': 'Product_Total_Profit',
    'Profit_mean': 'Product_Avg_Profit',
    'Quantity_sum': 'Product_Total_Quantity',
    'Quantity_mean': 'Product_Avg_Quantity',
    'Discount_mean': 'Product_Avg_Discount'
})

# Product popularity and profitability scores
product_features['Product_Popularity_Score'] = (
    product_features['Product_Transaction_Count'] /
    product_features['Product_Transaction_Count'].max() * 100
)
product_features['Product_Profitability_Score'] = (
    product_features['Product_Avg_Profit'] /
    product_features['Product_Avg_Profit'].max() * 100
)

# Merge back to main dataframe
df = df.merge(product_features, on='Product ID', how='left')


Creating product-level features...


In [ ]:
# 7. Create Regional Features
print("\nCreating regional features...")

regional_features = df.groupby('Country').agg({
    'Sales': ['sum', 'mean', 'count'],
    'Profit': ['sum', 'mean'],
    'Customer ID': 'nunique'
}).round(2)

regional_features.columns = ['_'.join(col).strip() for col in regional_features.columns.values]
regional_features = regional_features.rename(columns={
    'Sales_sum': 'Country_Total_Sales',
    'Sales_mean': 'Country_Avg_Sale',
    'Sales_count': 'Country_Transaction_Count',
    'Profit_sum': 'Country_Total_Profit',
    'Profit_mean': 'Country_Avg_Profit',
    'Customer ID_nunique': 'Country_Unique_Customers'
})

regional_features['Country_Market_Share'] = (
    regional_features['Country_Total_Sales'] /
    regional_features['Country_Total_Sales'].sum() * 100
)
regional_features['Country_Profit_Margin'] = (
    regional_features['Country_Total_Profit'] /
    regional_features['Country_Total_Sales'] * 100
)

# Merge back to main dataframe
df = df.merge(regional_features, on='Country', how='left')


Creating regional features...


In [ ]:
# 8. Feature Scaling
print("\nScaling numerical features...")

# Identify numerical columns (excluding ID columns and dates)
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
exclude_from_scaling = ['Row ID', 'Postal Code', 'Postal_Code_Missing',
                        'Is_Profitable', 'Is_Holiday_Season', 'Is_EndOfQuarter']
exclude_from_scaling += [col for col in df.columns if 'Encoded' in col or '_dummy' in col.lower()]

scaling_cols = [col for col in numerical_cols if col not in exclude_from_scaling]

# Use MinMaxScaler for better interpretability
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[scaling_cols] = scaler.fit_transform(df[scaling_cols])

# Save the scaler for future use
import joblib
joblib.dump(scaler, '/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/scaler.pkl')


Scaling numerical features...


['/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/scaler.pkl']

In [ ]:
# 9. Save Processed Data
print("\nSaving processed data...")

# Save full processed dataset
df_scaled.to_csv('/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/fully_processed.csv', index=False)

# Create aggregated datasets for different analyses
# Daily sales for time series forecasting
daily_sales = df.groupby('Order Date').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum',
    'Order ID': 'nunique'
}).reset_index()
daily_sales.columns = ['Date', 'Daily_Sales', 'Daily_Profit', 'Daily_Quantity', 'Daily_Orders']
daily_sales.to_csv('/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/daily_sales_agg.csv', index=False)

# Monthly sales by category
monthly_category = df.groupby([pd.Grouper(key='Order Date', freq='M'), 'Category']).agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).reset_index()
monthly_category.to_csv('/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/monthly_category.csv', index=False)

# Customer-level dataset for segmentation
customer_dataset = df.groupby('Customer ID').agg({
    'Customer_Recency': 'first',
    'Customer_Frequency': 'first',
    'Customer_Monetary': 'first',
    'Customer_Lifetime': 'first',
    'Customer_Total_Profit': 'first',
    'Country': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'Segment': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
}).reset_index()
customer_dataset.to_csv('/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/customer_segmentation.csv', index=False)

print("Processing complete! Files saved:")
print("/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/fully_processed.csv")
print("/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/daily_sales_agg.csv")
print("/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/monthly_category.csv")
print("/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/customer_segmentation.csv")


Saving processed data...
Processing complete! Files saved:
/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/fully_processed.csv
/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/daily_sales_agg.csv
/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/monthly_category.csv
/content/drive/MyDrive/GlobalSuperstore_Project/data/processed/customer_segmentation.csv


In [ ]:
print("\nFeature correlation with Profit:")
profit_corr = df.select_dtypes(include=np.number).corr()['Profit'].sort_values(ascending=False)
print("\nTop 10 Positive Correlations with Profit:")
print(profit_corr.head(10))
print("\nTop 10 Negative Correlations with Profit:")
print(profit_corr.tail(10))


Feature correlation with Profit:

Top 10 Positive Correlations with Profit:
Profit                         1.000000
Profit_Per_Unit                0.838825
Product_Avg_Profit             0.708139
Product_Profitability_Score    0.708139
Product_Total_Profit           0.604566
Sales                          0.484918
Unit_Price                     0.418647
Profit_Margin                  0.358106
Shipping Cost                  0.354441
Is_Profitable                  0.332106
Name: Profit, dtype: float64

Top 10 Negative Correlations with Profit:
Postal Code                 -0.009549
Product_Transaction_Count   -0.015632
Product_Popularity_Score    -0.015632
Market_Encoded              -0.017308
Row ID                      -0.019037
Customer_Recency            -0.025790
Sub-Category_Encoded        -0.040321
Country_Encoded             -0.053919
Product_Avg_Discount        -0.186730
Discount                    -0.316490
Name: Profit, dtype: float64
